In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%%capture
!pip install --upgrade datasets
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install --upgrade evaluate datasets
!pip install --upgrade transformers huggingface_hub

In [ ]:
import pandas as pd
import os

train_df = pd.read_csv('/content/drive/MyDrive/Twi_ASR/train.csv')
train_df = train_df.drop(index=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15761,229, 446, 653, 846,1035,	1216,	1316, 1410, 33062	])
test_df = pd.read_csv('/content/drive/MyDrive/Twi_ASR/test.csv')

In [4]:
path_list = []
for file_path in test_df['path']:
  new_path = os.path.splitext(file_path)[0] + ".wav"
  path_list.append(new_path)

test_df['path'] = path_list

In [5]:
path_list = []
for file_path in train_df['path']:
  new_path = os.path.splitext(file_path)[0] + ".wav"
  path_list.append(new_path)

train_df['path'] = path_list


In [6]:
train_df = train_df.iloc[1485:10000]

In [7]:
from datasets import load_dataset, Dataset

# Converting pandas DataFrame to Hugging Face Dataset
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)

In [8]:
train_df.loc[train_df['path'] == 'asante_twi_audio_2001.wav' ]

,path,sentence


In [9]:
from datasets import ClassLabel
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(test.remove_columns(["path"]), num_examples=20)

,sentence
0,Pia no bio
1,Na ɔkɔɔ ɔkodoɔ no baako a ɛyɛ Simon dea mu srɛɛ no sɛ ɔntwe mfiri fam kakra.
2,Ebi didi ebi akyi
3,"Na nna afa Petro ne wɔn a wɔka ne ho no; na wɔnyaneeɛ no, wɔhunuu n’animuonyam no ne mmarima baanu no sɛ wɔne no gyina hɔ."
4,Wote borɔfo
5,Medaase
6,Na ɔkɔe kowui
7,Hwɛ soro
8,ɛkɔm de me
9,A ohuu wɔ nkran


In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\)\(\/\•\’\—\·]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [12]:
train = train.map(remove_special_characters)
test = test.map(remove_special_characters)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [13]:
show_random_elements(test.remove_columns(['path']))

,sentence
0,mepaakyɛw ma sesa madwene
1,hwɛ soro
2,na yesu bua see no sɛ simon mewɔ asɛm bi meka mekyerɛ wo na ɔse kyerɛkyerɛfoɔ ka
3,medaase
4,akwadaa no to dwom
5,na ɔde ne nsa guu wɔn nyinaa so mmaako mmaako saa wɔn yadeɛ
6,merehwehwɛ yaw
7,merehwehwɛ yaw
8,yɛbehyia biom
9,mepaakyɛw spoken written mepawokyew


In [14]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[ń]', 'n', batch["sentence"])
    return batch

In [15]:
train = train.map(replace_hatted_characters)
test = test.map(replace_hatted_characters)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [17]:
vocab_train = train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train.column_names)
vocab_test = test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test.column_names)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [18]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [19]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'r': 0,
 'g': 1,
 's': 2,
 'k': 3,
 '0': 4,
 'w': 5,
 'o': 6,
 'l': 7,
 'ɛ': 8,
 'b': 9,
 'n': 10,
 'p': 11,
 'd': 12,
 'ɔ': 13,
 'e': 14,
 '4': 15,
 '1': 16,
 '5': 17,
 'y': 18,
 'c': 19,
 'f': 20,
 'h': 21,
 '9': 22,
 'm': 23,
 'i': 24,
 "'": 25,
 ' ': 26,
 'a': 27,
 'u': 28,
 't': 29,
 '2': 30}

In [20]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [21]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

33

In [22]:
import json
with open('/content/drive/MyDrive/Twi_ASR/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/Twi_ASR/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Preprocessing Audio Data


In [ ]:
"""from datasets import Dataset, concatenate_datasets

def process_dataset_in_chunks(dataset, chunk_size, fn, cache_file_name):
    num_chunks = len(dataset) // chunk_size + 1
    processed_chunks = []

    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(dataset))
        print(f"Processing chunk {i+1}/{num_chunks}")

        chunk = dataset.select(range(start, end))
        processed_chunk = chunk.map(fn, remove_columns=chunk.column_names, keep_in_memory=False, cache_file_name=cache_file_name, num_proc=1)
        processed_chunks.append(processed_chunk)

    return concatenate_datasets(processed_chunks)

# Process train and test datasets in chunks
chunk_size = 1000  # Adjust based on your RAM capacity
train_cache_file_name = "/content/drive/MyDrive/Twi_ASR/processed_train.arrow"
test_cache_file_name = "/content/drive/MyDrive/Twi_ASR/processed_test.arrow" """


In [ ]:
"""train = process_dataset_in_chunks(train, chunk_size, speech_file_to_array_fn, train_cache_file_name)
test = process_dataset_in_chunks(test, chunk_size, speech_file_to_array_fn, test_cache_file_name)"""

Processing chunk 1/34


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing chunk 2/34
Processing chunk 3/34
Processing chunk 4/34
Processing chunk 5/34
Processing chunk 6/34
Processing chunk 7/34
Processing chunk 8/34
Processing chunk 9/34
Processing chunk 10/34
Processing chunk 11/34
Processing chunk 12/34
Processing chunk 13/34
Processing chunk 14/34
Processing chunk 15/34
Processing chunk 16/34
Processing chunk 17/34
Processing chunk 18/34
Processing chunk 19/34
Processing chunk 20/34
Processing chunk 21/34
Processing chunk 22/34
Processing chunk 23/34
Processing chunk 24/34
Processing chunk 25/34
Processing chunk 26/34
Processing chunk 27/34
Processing chunk 28/34
Processing chunk 29/34
Processing chunk 30/34
Processing chunk 31/34
Processing chunk 32/34
Processing chunk 33/34
Processing chunk 34/34
Processing chunk 1/5


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Processing chunk 2/5
Processing chunk 3/5
Processing chunk 4/5
Processing chunk 5/5


In [28]:
import torchaudio
import os

# Define base path to the raw_audio folder
base_path = "/content/drive/MyDrive/Twi_ASR/processed_audio"

def speech_file_to_array_fn(batch):

    speech_array, sampling_rate = torchaudio.load(os.path.join(base_path, batch["path"]))
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]

    return batch

In [ ]:
train = train.map(speech_file_to_array_fn, remove_columns=train.column_names, num_proc=1)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

In [66]:
test = test.map(speech_file_to_array_fn, remove_columns=test.column_names, num_proc=1)

Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [67]:
import librosa
import numpy as np

def resample_audio(batch):
    """
    Resample audio data in 'speech' to 16kHz.
    """
    # Ensure 'speech' and 'sampling_rate' exist in batch
    if "speech" in batch and "sampling_rate" in batch:
        audio = np.asarray(batch["speech"])  # Convert speech to NumPy array
        orig_sr = batch["sampling_rate"]     # Original sampling rate

        # Perform resampling
        batch["speech"] = librosa.resample(y=audio, orig_sr=orig_sr, target_sr=16000)
        batch["sampling_rate"] = 16000  # Update sampling rate

    return batch

In [70]:
train = train.map(resample_audio, num_proc=1)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

In [71]:
test = test.map(resample_audio, num_proc=1)

Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [87]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train)-1)

print(train[rand_int]["target_text"])
ipd.Audio(data=np.asarray(train[rand_int]["speech"]), autoplay=True, rate=16000)

kɔɔ dua bi akyi 


In [88]:
rand_int = random.randint(0, len(train))

print("Target text:", train[rand_int]["target_text"])
print("Input array shape:", np.asarray(train[rand_int]["speech"]).shape)
print("Sampling rate:", train[rand_int]["sampling_rate"])

Target text: yɛ wɔ 
Input array shape: (18525,)
Sampling rate: 16000


In [89]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [91]:
train = train.map(prepare_dataset, remove_columns=train.column_names, batch_size=8, num_proc=1, batched=True)
test = test.map(prepare_dataset, remove_columns=test.column_names, batch_size=8, num_proc=1, batched=True)

Map:   0%|          | 0/8515 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4033 [00:00<?, ? examples/s]

In [92]:
# Save the processed datasets
train.save_to_disk("/content/drive/MyDrive/processed_dataset/train_dataset")
test.save_to_disk("/content/drive/MyDrive/processed_dataset/test_dataset")

Saving the dataset (0/4 shards):   0%|          | 0/8515 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/4033 [00:00<?, ? examples/s]

In [27]:
from datasets import load_from_disk

# Load the datasets back
train = load_from_disk("/content/drive/MyDrive/processed_dataset/train_dataset")
test = load_from_disk("/content/drive/MyDrive/processed_dataset/test_dataset")

In [ ]:
train.to_csv('/content/drive/MyDrive/Twi_ASR/2_processed_dataset/csv/processed_train.csv')
test.to_csv('/content/drive/MyDrive/Twi_ASR/2_processed_dataset/csv/processed_test.csv')

Creating CSV from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

656685

**Setting-up Trainer**

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
from evaluate import load

wer_metric = load("wer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [32]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Handle special tokens in labels
    labels = np.copy(pred.label_ids)
    labels[labels == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and references
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(labels)

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


"""def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}"""

'def compute_metrics(pred):\n    pred_logits = pred.predictions\n    pred_ids = np.argmax(pred_logits, axis=-1)\n\n    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id\n\n    pred_str = processor.batch_decode(pred_ids)\n    # we do not want to group tokens when computing the metrics\n    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)\n\n    wer = wer_metric.compute(predictions=pred_str, references=label_str)\n\n    return {"wer": wer}'

In [33]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2178: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/wav2vec2-large-xlsr-twi",  # Ensure output directory is correct
    group_by_length=True,
    per_device_train_batch_size=8,  # Reduced batch size for stability
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=False,  # Disabled fp16 to avoid potential NaN issues
    save_steps=1000,  # Save model every 1000 steps instead of 400
    eval_steps=1000,  # Same for evaluation steps
    logging_steps=400,  # Log every 400 steps
    learning_rate=2e-5,  # Lower learning rate for stability
    warmup_steps=500,
    save_total_limit=2,  # Keep only 2 saved checkpoints
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)

<ipython-input-36-23c800d7e7f4>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
